In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
from sklearn.metrics import make_scorer
from sklearn.metrics import mean_squared_log_error

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list the files in the input directory
import xgboost as xgb
from sklearn import preprocessing
import os
from scipy.stats import skew
from sklearn.model_selection import train_test_split
print(os.listdir("../input"))
from sklearn.linear_model import Ridge, RidgeCV, ElasticNet, LassoCV, LassoLarsCV
from sklearn.model_selection import cross_val_score

from matplotlib import pyplot as plt
from sklearn.svm import SVC
from sklearn.model_selection import GridSearchCV, cross_val_score, KFold
# Any results you write to the current directory are saved as output.

['test_9K3DBWQ.csv', 'Data_Dictionary.xlsx', 'train.csv']


In [2]:
import math
def rmsle(y, y_pred):
    assert len(y) == len(y_pred)
    to_sum = (np.log1p(y_pred)-np.log1p(y) ** 2.0)
    return (sum(to_sum) * (1.0/len(y))) ** 0.5

In [3]:
train_data = pd.read_csv("../input/train.csv")
y_train = train_data[train_data.columns[-1]]
del train_data[train_data.columns[-1]]
#train_data, cv_data, y_train, y_cv = train_test_split(train_data, y, test_size=0.20, random_state=42)
test_data = pd.read_csv("../input/test_9K3DBWQ.csv")

In [4]:
train_data.head()

,id,account_type,gender,age,region_code,cc_cons_apr,dc_cons_apr,cc_cons_may,dc_cons_may,cc_cons_jun,dc_cons_jun,cc_count_apr,cc_count_may,cc_count_jun,dc_count_apr,dc_count_may,dc_count_jun,card_lim,personal_loan_active,vehicle_loan_active,personal_loan_closed,vehicle_loan_closed,investment_1,investment_2,investment_3,investment_4,debit_amount_apr,credit_amount_apr,debit_count_apr,credit_count_apr,max_credit_amount_apr,debit_amount_may,credit_amount_may,credit_count_may,debit_count_may,max_credit_amount_may,debit_amount_jun,credit_amount_jun,credit_count_jun,debit_count_jun,max_credit_amount_jun,loan_enq,emi_active
0,23334,saving,M,63,305,7695.00,NaN,479.00,NaN,622.0,NaN,2.0,1.0,3.0,NaN,NaN,NaN,150000.0,NaN,NaN,NaN,NaN,NaN,118845.2,50000.0,NaN,51205.47,50259.0,3.0,1.0,50000.0,24740.22,114252.0,3.0,2.0,45000.0,20215.85,1279.37,7.0,3.0,470.0,NaN,0.00
1,20329,current,M,33,326,52702.00,988.72,29873.00,NaN,1087.0,2806.24,11.0,14.0,11.0,1.0,NaN,6.0,176000.0,NaN,NaN,1.0,NaN,NaN,NaN,NaN,NaN,159050.76,169812.0,20.0,9.0,70000.0,103102.85,174606.0,6.0,10.0,150000.0,122037.66,55390.00,8.0,19.0,27000.0,NaN,4963.33
2,17679,current,M,44,264,10800.49,3215.52,21659.64,1607.0,1168.0,355.00,6.0,9.0,6.0,3.0,2.0,14.0,65000.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,51484.55,38942.0,16.0,4.0,11700.0,15283.00,23051.0,2.0,7.0,13800.0,46827.00,29297.91,6.0,19.0,20800.0,NaN,0.00
3,16428,saving,M,44,233,20936.00,NaN,2094.00,NaN,11778.5,NaN,9.0,6.0,6.0,NaN,NaN,NaN,600000.0,NaN,NaN,NaN,NaN,NaN,132442.0,NaN,NaN,1981.07,2294.0,2.0,1.0,2000.0,19361.24,19529.0,1.0,1.0,19300.0,5426.55,7544.00,3.0,1.0,5300.0,NaN,1238.18
4,18721,saving,F,35,324,1183.83,NaN,3656.29,NaN,945.0,NaN,2.0,7.0,9.0,NaN,NaN,NaN,100000.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,6462.00,10271.0,6.0,1.0,10000.0,7461.00,3194.0,2.0,3.0,3000.0,26178.05,53483.00,4.0,4.0,45000.0,NaN,0.00


In [5]:
test_data.head()

,id,account_type,gender,age,region_code,cc_cons_apr,dc_cons_apr,cc_cons_may,dc_cons_may,cc_cons_jun,dc_cons_jun,cc_count_apr,cc_count_may,cc_count_jun,dc_count_apr,dc_count_may,dc_count_jun,card_lim,personal_loan_active,vehicle_loan_active,personal_loan_closed,vehicle_loan_closed,investment_1,investment_2,investment_3,investment_4,debit_amount_apr,credit_amount_apr,debit_count_apr,credit_count_apr,max_credit_amount_apr,debit_amount_may,credit_amount_may,credit_count_may,debit_count_may,max_credit_amount_may,debit_amount_jun,credit_amount_jun,credit_count_jun,debit_count_jun,max_credit_amount_jun,loan_enq,emi_active
0,6975,saving,M,36,447,17942.70,1170.0,1656.00,NaN,4551.00,NaN,6.0,2.0,6.0,1.0,NaN,NaN,75000.0,NaN,NaN,NaN,1.0,NaN,NaN,NaN,NaN,10806.93,25279.0,7.0,2.0,20000.0,21530.34,15416.52,3.0,7.0,10000.0,17316.44,5154.0,2.0,8.0,5000.0,NaN,0.00
1,43223,current,M,31,233,44870.36,NaN,34177.00,NaN,25889.21,NaN,13.0,19.0,36.0,NaN,NaN,NaN,116000.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.00
2,45502,current,M,26,823,1073.00,2442.0,4141.00,390.0,6740.29,642.0,3.0,5.0,2.0,8.0,3.0,3.0,40000.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,83205.22,105813.0,33.0,5.0,39000.0,90312.48,65227.00,4.0,22.0,26926.0,33981.00,37762.0,3.0,15.0,29352.0,NaN,2885.82
3,12368,current,M,39,878,33060.58,NaN,3797.12,NaN,291.00,NaN,16.0,12.0,15.0,NaN,NaN,NaN,86000.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,45342.25,59640.0,4.0,5.0,28753.0,84699.30,63393.00,4.0,8.0,57185.0,52927.00,89425.0,7.0,11.0,32696.0,NaN,0.00
4,40415,current,M,44,404,18236.90,NaN,30513.30,NaN,20395.28,NaN,42.0,38.0,20.0,NaN,NaN,NaN,250000.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,25493.95,25154.0,4.0,1.0,25000.0,39813.40,45120.00,1.0,5.0,45000.0,31656.69,34275.0,3.0,4.0,19000.0,NaN,3342.58


In [6]:
le = preprocessing.LabelEncoder()
train_data['account_type'] = le.fit_transform(train_data['account_type'])
#cv_data['account_type'] = le.transform(cv_data['account_type'])
test_data['account_type'] = le.transform(test_data['account_type'])

train_data['gender'] = le.fit_transform(train_data['gender'])
#cv_data['gender'] = le.transform(cv_data['gender'])
test_data['gender'] = le.transform(test_data['gender'])

train_data['loan_enq'] = train_data['loan_enq'].astype(str, inplace = True)
#cv_data['loan_enq'] = cv_data['loan_enq'].astype(str, inplace = True)
test_data['loan_enq'] = test_data['loan_enq'].astype(str, inplace = True)

train_data['loan_enq'] = le.fit_transform(train_data['loan_enq'])
#cv_data['loan_enq'] = le.transform(cv_data['loan_enq'])
test_data['loan_enq'] = le.transform(test_data['loan_enq'])

In [7]:
all_data = pd.concat((train_data, test_data), axis = 0, sort = False)

In [8]:
train_data.head()

,id,account_type,gender,age,region_code,cc_cons_apr,dc_cons_apr,cc_cons_may,dc_cons_may,cc_cons_jun,dc_cons_jun,cc_count_apr,cc_count_may,cc_count_jun,dc_count_apr,dc_count_may,dc_count_jun,card_lim,personal_loan_active,vehicle_loan_active,personal_loan_closed,vehicle_loan_closed,investment_1,investment_2,investment_3,investment_4,debit_amount_apr,credit_amount_apr,debit_count_apr,credit_count_apr,max_credit_amount_apr,debit_amount_may,credit_amount_may,credit_count_may,debit_count_may,max_credit_amount_may,debit_amount_jun,credit_amount_jun,credit_count_jun,debit_count_jun,max_credit_amount_jun,loan_enq,emi_active
0,23334,1,1,63,305,7695.00,NaN,479.00,NaN,622.0,NaN,2.0,1.0,3.0,NaN,NaN,NaN,150000.0,NaN,NaN,NaN,NaN,NaN,118845.2,50000.0,NaN,51205.47,50259.0,3.0,1.0,50000.0,24740.22,114252.0,3.0,2.0,45000.0,20215.85,1279.37,7.0,3.0,470.0,1,0.00
1,20329,0,1,33,326,52702.00,988.72,29873.00,NaN,1087.0,2806.24,11.0,14.0,11.0,1.0,NaN,6.0,176000.0,NaN,NaN,1.0,NaN,NaN,NaN,NaN,NaN,159050.76,169812.0,20.0,9.0,70000.0,103102.85,174606.0,6.0,10.0,150000.0,122037.66,55390.00,8.0,19.0,27000.0,1,4963.33
2,17679,0,1,44,264,10800.49,3215.52,21659.64,1607.0,1168.0,355.00,6.0,9.0,6.0,3.0,2.0,14.0,65000.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,51484.55,38942.0,16.0,4.0,11700.0,15283.00,23051.0,2.0,7.0,13800.0,46827.00,29297.91,6.0,19.0,20800.0,1,0.00
3,16428,1,1,44,233,20936.00,NaN,2094.00,NaN,11778.5,NaN,9.0,6.0,6.0,NaN,NaN,NaN,600000.0,NaN,NaN,NaN,NaN,NaN,132442.0,NaN,NaN,1981.07,2294.0,2.0,1.0,2000.0,19361.24,19529.0,1.0,1.0,19300.0,5426.55,7544.00,3.0,1.0,5300.0,1,1238.18
4,18721,1,0,35,324,1183.83,NaN,3656.29,NaN,945.0,NaN,2.0,7.0,9.0,NaN,NaN,NaN,100000.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,6462.00,10271.0,6.0,1.0,10000.0,7461.00,3194.0,2.0,3.0,3000.0,26178.05,53483.00,4.0,4.0,45000.0,1,0.00


In [9]:
#log transform the target:
y_train = np.log1p(y_train)

#log transform skewed numeric features which don't have more than 75% data missing
numeric_feats_all = train_data.dtypes[(train_data.dtypes != "object")].index

numeric_feats = [feature for feature in numeric_feats_all if(train_data[feature].isna().sum()<12000)]
print(numeric_feats)

['id', 'account_type', 'gender', 'age', 'region_code', 'cc_cons_apr', 'cc_cons_may', 'cc_cons_jun', 'cc_count_apr', 'cc_count_may', 'cc_count_jun', 'card_lim', 'debit_amount_apr', 'credit_amount_apr', 'debit_count_apr', 'credit_count_apr', 'max_credit_amount_apr', 'debit_amount_may', 'credit_amount_may', 'credit_count_may', 'debit_count_may', 'max_credit_amount_may', 'debit_amount_jun', 'credit_amount_jun', 'credit_count_jun', 'debit_count_jun', 'max_credit_amount_jun', 'loan_enq', 'emi_active']


In [10]:
skewed_feats = train_data[numeric_feats].apply(lambda x: skew(x)) #compute skewness

skewed_feats = skewed_feats[skewed_feats > 0.75]
skewed_feats = skewed_feats.index
print(skewed_feats)
train_data[skewed_feats] = np.log1p(train_data[skewed_feats])
test_data[skewed_feats] = np.log1p(test_data[skewed_feats])

Index(['account_type', 'age', 'cc_cons_apr', 'cc_cons_may', 'cc_cons_jun',
       'emi_active'],
      dtype='object')


In [11]:
def handle_missing_values(train_data, test_data, feature, val = 0):
    train_data[feature].replace(to_replace = np.nan, value = val, inplace = True)  
    test_data[feature].replace(to_replace = np.nan, value = val, inplace = True)
    return train_data, test_data
    
for feature in train_data.columns:
    train_data, test_data = handle_missing_values(train_data, test_data, feature, 0)

In [12]:
# for feature in object_feats:
#     train_data[feature].replace(to_replace = np.nan, value = 0, inplace = True)  
#     test_data[feature].replace(to_replace = np.nan, value = val, inplace = True)

In [13]:
print(train_data.isnull().values.any())   # prints False now :)
print(test_data.isnull().values.any())   # prints False now :)

False
False


In [14]:
from sklearn import datasets
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import GradientBoostingRegressor, RandomForestRegressor, ExtraTreesRegressor, VotingRegressor



# Training classifiers
reg1 = GradientBoostingRegressor(random_state=1, n_estimators=300)
reg2 = RandomForestRegressor(random_state=1, n_estimators=300)
reg3 = ExtraTreesRegressor(random_state=1, n_estimators=300)
reg4 = LinearRegression()
ereg = VotingRegressor(estimators=[('gb', reg1), ('rf', reg2), ('et', reg3), ('lr', reg4)])
ereg = ereg.fit(train_data, y_train)

In [15]:
def make_test_pred(best_estimator):
    test_pred = best_estimator.predict(test_data)
    test_pred = np.expm1(test_pred)
    test_submission = pd.DataFrame()
    test_submission['id'] = test_data['id']
    test_submission['cc_cons'] = test_pred
    test_submission.to_csv('combined.csv', index=False)                 

In [16]:
make_test_pred(ereg)